In [21]:
# Product Class
class Product:
    def __init__(self, name, price, quantity, product_id):
        self.name = name
        self.price = price
        self.quantity = quantity
        self.product_id = product_id

    def __str__(self):
        return f"ID: {self.product_id}, {self.name} - ${self.price}, Quantity: {self.quantity}"

# Linked List Node
class ProductNode:
    def __init__(self, product):
        self.product = product
        self.next = None

# Linked List Class
class ProductLinkedList:
    def __init__(self):
        self.head = None
        self.tail = None

    def insert_product(self, product):
        temp = self.head
        while temp:
            if temp.product.name == product.name:
                temp.product.quantity += product.quantity
                print(f" Restocked '{product.name}'. New quantity: {temp.product.quantity}")
                return
            temp = temp.next

        new_node = ProductNode(product)
        if self.head is None:
            self.head = self.tail = new_node
        else:
            self.tail.next = new_node
            self.tail = new_node

    def delete_product(self, name, quantity=1):
        temp = self.head
        prev = None
        while temp:
            if temp.product.name == name:
                if quantity <= 0:
                    print(" Invalid quantity specified. Must be greater than 0.")
                    return

                if temp.product.quantity == 0:
                    print(f" Product '{name}' is out of stock.")
                    return

                if quantity > temp.product.quantity:
                    raise ValueError(f" Cannot delete {quantity} items. Only {temp.product.quantity} in stock for '{name}'.")

                temp.product.quantity -= quantity

                if temp.product.quantity == 0:
                    if prev:
                        prev.next = temp.next
                    else:
                        self.head = temp.next
                    if temp == self.tail:
                        self.tail = prev
                    print(f" Product '{name}' removed as quantity reached 0.")
                else:
                    print(f" Decreased quantity of '{name}' by {quantity}. New quantity: {temp.product.quantity}")
                return
            prev = temp
            temp = temp.next

        print(f" Product '{name}' not found.")

    def bubble_sort(self, attribute):
        if self.head is None or self.head.next is None:
            return

        swapped = True
        while swapped:
            swapped = False
            current = self.head

            while current and current.next:
                if (attribute == "name" and (current.product.name or "") > (current.next.product.name or "")) or \
                   (attribute == "price" and (current.product.price or 0) > (current.next.product.price or 0)):
                    current.product, current.next.product = current.next.product, current.product
                    swapped = True
                current = current.next

        print(f" Products sorted by {attribute}")
        self.display()

    def filter_by_price(self, min_price, max_price):
        temp = self.head
        found = False
        print(f"\n Products in the price range ${min_price} - ${max_price}:")
        while temp:
            if min_price <= temp.product.price <= max_price:
                print(f"  - {temp.product}")
                found = True
            temp = temp.next
        if not found:
            print(" No products found in this price range.")

    def search_product(self, name):
        temp = self.head
        while temp:
            if temp.product.name == name:
                print(f" Product found: {temp.product}")
                return temp.product
            temp = temp.next
        print(f" Product '{name}' not found.")
        return None

    def display(self):
        if self.head is None:
            print(" No products in this category.")
            return
        temp = self.head
        while temp:
            print(f"  - {temp.product}")
            temp = temp.next

# BST Node
class BSTNode:
    def __init__(self, category_id, category_name):
        self.category_id = category_id
        self.category_name = category_name
        self.products = ProductLinkedList()
        self.left = None
        self.right = None

# BST Class
class ProductBST:
    def __init__(self):
        self.root = None
        self.product_count = {}  # Track number of products per category

    def insert_category(self, category_id, category_name):
        if self.find_category(category_name) is None:
            self.root = self._insert_category_recursive(self.root, category_id, category_name)

    def _insert_category_recursive(self, root, category_id, category_name):
        if root is None:
            return BSTNode(category_id, category_name)
        if category_name < root.category_name:
            root.left = self._insert_category_recursive(root.left, category_id, category_name)
        elif category_name > root.category_name:
            root.right = self._insert_category_recursive(root.right, category_id, category_name)
        return root

    def delete_category(self, category_name):
        self.root = self._delete_category_recursive(self.root, category_name)

    def _delete_category_recursive(self, root, category_name):
        if root is None:
            return root
        if category_name < root.category_name:
            root.left = self._delete_category_recursive(root.left, category_name)
        elif category_name > root.category_name:
            root.right = self._delete_category_recursive(root.right, category_name)
        else:
            if root.left is None:
                return root.right
            elif root.right is None:
                return root.left
            min_larger_node = self._find_min(root.right)
            root.category_id, root.category_name, root.products = min_larger_node.category_id, min_larger_node.category_name, min_larger_node.products
            root.right = self._delete_category_recursive(root.right, min_larger_node.category_name)
        return root

    def find_category(self, category_name):
        return self._find_category_recursive(self.root, category_name)

    def _find_category_recursive(self, root, category_name):
        if root is None or root.category_name == category_name:
            return root
        if category_name < root.category_name:
            return self._find_category_recursive(root.left, category_name)
        return self._find_category_recursive(root.right, category_name)

    def _find_min(self, node):
        current = node
        while current.left is not None:
            current = current.left
        return current

    def add_product(self, category_id, category_name, product_name, price, quantity):
        if category_name not in self.product_count:
            self.insert_category(category_id, category_name)
            self.product_count[category_name] = 1
        else:
            self.product_count[category_name] += 1

        product_id = category_id * 100 + self.product_count[category_name]
        product = Product(product_name, price, quantity, product_id)

        category_node = self.find_category(category_name)
        if category_node is not None:
            category_node.products.insert_product(product)
        else:
            print(f"Error: Category '{category_name}' could not be found after insertion.")

    def delete_product(self, category_name, product_name, quantity=1):
        category_node = self.find_category(category_name)
        if category_node:
            category_node.products.delete_product(product_name, quantity)

    def search_product(self, category_name, product_name):
        category_node = self.find_category(category_name)
        if category_node:
            return category_node.products.search_product(product_name)
        print(f" Category '{category_name}' not found.")
        return None

    def sort_products(self, category_name, attribute):
        category_node = self.find_category(category_name)
        if category_node:
            category_node.products.bubble_sort(attribute)

    def filter_by_price(self, category_name, min_price, max_price):
        category_node = self.find_category(category_name)
        if category_node:
            category_node.products.filter_by_price(min_price, max_price)

    def display(self):
        self._inorder_traversal(self.root)

    def _inorder_traversal(self, root):
        if root:
            self._inorder_traversal(root.left)
            print(f"\n Category ID: {root.category_id}, Name: {root.category_name}")
            root.products.display()
            self._inorder_traversal(root.right)

# Main Execution
if __name__ == "__main__":
    bst = ProductBST()

    # Smartphones (101)
    bst.add_product(101, "Smartphone", "iPhone 14", 999, 5)
    bst.add_product(101, "Smartphone", "Galaxy S22", 799, 3)
    bst.add_product(101, "Smartphone", "Pixel 7", 699, 4)

    # Laptops (102)
    bst.add_product(102, "Laptop", "MacBook Pro", 1999, 2)
    bst.add_product(102, "Laptop", "Dell XPS", 1499, 3)
    bst.add_product(102, "Laptop", "HP Spectre", 1299, 4)

    # Tablets (103)
    bst.add_product(103, "Tablet", "iPad Air", 599, 6)
    bst.add_product(103, "Tablet", "Galaxy Tab", 499, 5)
    bst.add_product(103, "Tablet", "Surface Go", 649, 2)

    # Headphones (104)
    bst.add_product(104, "Headphones", "Sony WH-1000XM4", 349, 5)
    bst.add_product(104, "Headphones", "Bose QC45", 329, 4)
    bst.add_product(104, "Headphones", "AirPods Max", 549, 3)
    # Cameras (105)
    bst.add_product(105, "Camera", "Canon EOS R10", 1099, 2)
    bst.add_product(105, "Camera", "Nikon Z50", 1199, 2)
    bst.add_product(105, "Camera", "Sony Alpha a6400", 1249, 1)

    # Smartwatches (106)
    bst.add_product(106, "Smartwatch", "Apple Watch Series 8", 499, 4)
    bst.add_product(106, "Smartwatch", "Samsung Galaxy Watch 5", 399, 5)
    bst.add_product(106, "Smartwatch", "Fitbit Sense 2", 329, 6)

    # Monitors (107)
    bst.add_product(107, "Monitor", "Dell Ultrasharp", 529, 2)
    bst.add_product(107, "Monitor", "LG UltraFine", 599, 3)
    bst.add_product(107, "Monitor", "ASUS ProArt", 649, 1)


    # Demonstrate all methods
    bst.delete_product("Tablet", "Galaxy Tab", 2)

    bst.search_product("Smartphone", "iPhone 14")

    bst.sort_products("Laptop", "price")

    bst.filter_by_price("Smartwatch", 300, 500)

    print("\n Final Inventory with Category and Product IDs:")
    bst.display()


 Decreased quantity of 'Galaxy Tab' by 2. New quantity: 3
 Product found: ID: 10101, iPhone 14 - $999, Quantity: 5
 Products sorted by price
  - ID: 10203, HP Spectre - $1299, Quantity: 4
  - ID: 10202, Dell XPS - $1499, Quantity: 3
  - ID: 10201, MacBook Pro - $1999, Quantity: 2

 Products in the price range $300 - $500:
  - ID: 10601, Apple Watch Series 8 - $499, Quantity: 4
  - ID: 10602, Samsung Galaxy Watch 5 - $399, Quantity: 5
  - ID: 10603, Fitbit Sense 2 - $329, Quantity: 6

 Final Inventory with Category and Product IDs:

 Category ID: 105, Name: Camera
  - ID: 10501, Canon EOS R10 - $1099, Quantity: 2
  - ID: 10502, Nikon Z50 - $1199, Quantity: 2
  - ID: 10503, Sony Alpha a6400 - $1249, Quantity: 1

 Category ID: 104, Name: Headphones
  - ID: 10401, Sony WH-1000XM4 - $349, Quantity: 5
  - ID: 10402, Bose QC45 - $329, Quantity: 4
  - ID: 10403, AirPods Max - $549, Quantity: 3

 Category ID: 102, Name: Laptop
  - ID: 10203, HP Spectre - $1299, Quantity: 4
  - ID: 10202, Dell X

| Method                                   | Time complexity            |
| -----------------------------------------|----------------------------|                       
| insert_product(linked list)              | O(n)                       |  n=no.of products in linked list
| delete_product(linked list)              | O(n)                       |  n=no.of products in linked list
| bubble_sort(linked list)                 | O(n^2)                     |
| filter_by_price                          | O(n)                       |
| search_product                           | O(n)                       |  
| display                                  | O(n)                       |
| _insert_category_recursive               | O(h)                       | h = height of the tree
| _delete_category_recursive               | O(h)                       | 
| _find_category_recursive                 | O(h)                       |
| add_product                              | O(1)                       |
| delete_product                           | O(n)                       |
| search_product                           | O(h + n)                   | h = height of the tree,n =  no.of products in linked list
| _inorder_traversal                       | O(n)                       | n = no.of nodes